In [ ]:
# PL 01 Gonçalo Iglésias / José Silva / André Amorim
# CESAE. Outubro 2023

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementClickInterceptedException
import time
import mysql.connector

In [2]:
service = Service()
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
options.add_experimental_option('excludeSwitches', ['enable-logging'])

In [3]:
# abrir o website
driver = webdriver.Chrome(service=service, options=options)
driver.get('https://worten.pt/')
time.sleep(5)

In [4]:
# aceitar os cookies
acceptCookies = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div/section/footer/button[2]')
acceptCookies.click()
time.sleep(5)

In [5]:
# encontrar a caixa de pesquisa
searchBar = driver.find_element(By.NAME, 'query')

In [6]:
# pesquisar por iphone 15
searchBar.send_keys("iphone 15")
searchBar.send_keys(Keys.ENTER)
time.sleep(5)

In [7]:
# filtrar resultados de pesquisa para obter apenas Telemóveis e Smartphones > iPhone
checkIphoneBox = driver.find_element(By.XPATH, '//*[@id="accordion__content-_cc2"]/div/div/ul/li[1]/ul/li/div/label')
checkIphoneBox.click()

In [8]:
# funções para extrair os dados dos iphone 15 e variáveis globais necessárias para correr as funções
def getProductImage():
  imgGrid = driver.find_elements(By.CSS_SELECTOR, 'header.product-card__header figure.product-card__image-link img')
  for img in imgGrid:
    productImageList.append(img.get_attribute('src'))
  return productImageList

def getProductURL():
  urlGrid = driver.find_elements(By.CSS_SELECTOR, 'div.product-card--grid-container a')
  excludingSuggestionsUrl = [url for url in urlGrid if url.get_attribute("class") != "related-cards__card"]
  for url in excludingSuggestionsUrl:
    productUrlList.append(url.get_attribute('href'))
  return productUrlList

def getProductName():
  nameGrid = driver.find_elements(By.CLASS_NAME, 'produc-card__name__link')
  for name in nameGrid:
    productNameList.append(name.text)
  return productNameList

def getProductPrice():
  priceGrid = driver.find_elements(By.CSS_SELECTOR, 'div.product-card--grid-container span.value')
  excludingSuggestionsPrice = [price for price in priceGrid if price.get_attribute("class") != "related-cards__card"]
  for price in excludingSuggestionsPrice:
    productPriceList.append(price.text)
  return productPriceList

productImageList = [] 
productUrlList = []
productNameList = []
productPriceList = []

# navega no website e faz o scrape da informação das páginas de resultados
while True:
  try:
    productImageList = getProductImage() 
    productUrlList = getProductURL()
    productNameList = getProductName()
    productPriceList = getProductPrice()
    # passa para a página seguinte
    nextPage = driver.find_element(By.XPATH, '//*[@id="filtered-grid"]/div/div/section/div/div[1]/div[2]/div/div/ul[1]/li[2]/a')
    nextPage.click() 
    # exceção assegura que se sai do while loop quando o botão existe mas está com status = disabled
  except ElementClickInterceptedException:
        break

In [9]:
# adicionar os resultados obtidos a uma lista de dicionários com as chaves: image, url, name, price
def addPhonestoList():
  for i in range(len(productImageList)):
    iphone = {
      "image": productImageList[i],
      "url": productUrlList[i],
      "name": productNameList[i],
      "price": productPriceList[i]
    }
    iphonesList.append(iphone)
  return iphonesList

iphonesList = []
iphonesList = addPhonestoList()

In [ ]:
# converter o preço de string para float, para ser possível guardar na BD
def convertPrice(iphonesList):
    for phone in iphonesList:
        phone["price"] = phone["price"].replace(",",".")
        phone['price'] = float(phone['price'])
    return iphonesList

iphonesList = convertPrice(iphonesList)

In [23]:
# ligar ao servidor MySQL
conn = mysql.connector.connect(user = 'root', host = 'localhost', autocommit=True)
# cursor para executar comandos na BD
cursorObject = conn.cursor()
# criar a base de dados com o nome wortendb
cursorObject.execute("CREATE DATABASE IF NOT EXISTS wortendb")
# usar a base de dados wortendb
cursorObject.execute("USE wortendb")
# criar a tabela iphones
cursorObject.execute('''
    CREATE TABLE IF NOT EXISTS iphones (id INT NOT NULL AUTO_INCREMENT, 
    image VARCHAR(255) NOT NULL, 
    url VARCHAR(255) NOT NULL, 
    name VARCHAR(255) NOT NULL,
    price DECIMAL(10,2) NOT NULL, 
    PRIMARY KEY (id))
    ''')

# criar uma lista de listas para adicionar os valores à base de dados wortendb
values = []
for iphone in iphonesList:
  iphoneData = [iphone["image"], iphone["url"], iphone["name"], iphone["price"]]
  values.append(iphoneData)

# usar o executemany para inserir de uma só vez todos os elementos da lista values (usa menos memória)
write = "INSERT INTO iphones (image, url, name, price) VALUES (%s, %s, %s, %s)"
cursorObject.executemany(write, values)
  
# encerrar a ligação ao servidor MySQL
conn.close()